In [147]:
import matplotlib as plt
import seaborn as sns
import numpy as np
import random
import os
from torch.utils.data import DataLoader, TensorDataset, Dataset
import dill

import pandas as pd
from tqdm import tqdm
import sqlalchemy

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.optim import lr_scheduler

In [148]:
class CFG:
    '''

    All hyperparameters are here

    '''
    batch_size = 8192

    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    # device='cpu'

In [149]:
def set_seed(seed=42):
    '''
    
    Sets the seed of the entire notebook so results are the same every time we run.
    This is for REPRODUCIBILITY.

    '''
    np.random.seed(seed)
    random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    # When running on the CuDNN backend, two further options must be set
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    # Set a fixed value for the hash seed
    os.environ['PYTHONHASHSEED'] = str(seed)
    
set_seed()

In [150]:
path = "/Users/avarlamov/Downloads/Full_16_09_22.csv"

In [151]:
id_features = [
    "root_id",
    "version_id",
    "request_number",
    "mos_ru_request_number",
    "root_identificator_of_maternal",
    "number_of_maternal",
    "deffect_category_id",
    "deffect_id",
    "district_code",
    "hood_code",
    "performing_company",
    "inn_of_performing_company",
    "id_of_reason_for_decline", # all values are NaN in taken sample
    "id_of_reason_for_decline_of_org",   # all values are NaN in taken sample
    "id_work_type_done",
    "id_guarding_events"
]

date_features = [
    "date_of_creation",
    "date_of_start",
    "wanted_time_from",
    "wanted_time_to",
    "date_of_review",
    "date_of_last_return_for_revision",
    "closure_date"
]

# All numerical features are discrete -> casting to Int64
numerical_features = [
    "times_returned",    # might be useful
    "adress_unom"
]

categorical_features = [
    "name_of_source",
    "name_of_source_eng",
    "name_of_creator",
    "role_of_user",
    "deffect_category_name",
    "deffect_category_name_eng",
    "deffect_name",
    "short_deffect_name",
    "need_for_revision",
    "urgency_category",
    "urgency_category_eng",
    "district",
    "hood",
    "adress_of_problem",
    "incident_feature",
    "serving_company",
    "dispetchers_number",
    "request_status",
    "request_status_eng",
    "reason_for_decline",   # all values are NaN in taken sample
    "reason_for_decline_of_org", # all values are NaN in taken sample
    "efficiency",
    "efficiency_eng",
    "being_on_revision",
    "alerted_feature",
    "grade_for_service",
    "grade_for_service_eng",
    "payment_category",
    "payment_category_eng",
    "payed_by_card"
]

# String features can also be categorical with a lot of possible values
string_features = [
    "last_name_redacted",
    "commentaries", # might be useful
    "code_of_deffect",
    "description",  # might be useful
    "presence_of_question", # might be useful

    "dispetchers_number",
    "owner_company",
    "work_type_done", # might be useful
    "used_material",
    "guarding_events", # might be useful
    "review",    # might be useful - here is the information about the results of the work

    # These are numerical features, that include some strangely filled rows:
    "floor",
    "flat_number",
    "porch",
]

features = string_features + date_features + numerical_features + categorical_features + id_features

In [152]:
useful_features_for_nlp = [
    "commentaries", "description", "presence_of_question", "work_type_done", "guarding_events", "review"
    ]

useful_features = [
    "need_for_revision",
    "times_returned",
    "request_status", 
    "efficiency",
    "date_of_creation",
    "date_of_start",
    "closure_date",
    "deffect_name",
    "deffect_category_id"
    ]

In [153]:
def change_columns(df, naming_path="naming.csv") -> pd.DataFrame:
    '''
    Function which changes column names for pd.DataFrame
    '''
    
    new_names = list(
        map(
            lambda x: x.strip(),
            list(pd.read_csv(naming_path)['new_name'])
        )
    )
    df.columns = new_names

    return df

# Изменение типов колонок:
def cast_types(df) -> pd.DataFrame:

    to_str = string_features + categorical_features + id_features

    for feature in to_str:
        df = df.astype({
            feature: "object"
        })

    for feature in numerical_features:
        df = df.astype({
            feature: "Int64"
        })

    for feature in date_features:
        df = df.astype({
            feature: "datetime64[ns]"   # if the time precission is given up to ns - that's the format
        })
    
    return df

def transform_df(df, naming_path="naming.csv"):
    df = cast_types(change_columns(df))

    return df

In [154]:
def get_all_unique_values(df_iterator, feature) -> list:
    '''
    Функция возвращающая все уникальные значения для колонки
    '''
    values = set()
    for df in tqdm(df_iterator, desc="Iter: "):
        df = cast_types(change_columns(df))
        values = values.union(
            set(df[feature].unique())
        )

    return list(values)

---

# Построение модели:

In [155]:
import datetime as DT
from datetime import datetime

In [156]:
test_dic = {
    "root_id": "73417411",
    "version_id": "73509990",
    "request_number": "00002335/21",
    "mos_ru_request_number": None,
    "name_of_source": "телефон",
    "name_of_source_eng": "phone",
    "name_of_creator": "frontline",
    "incident_feature": "Нет",
    "root_identificator_of_maternal": None,
    "number_of_maternal": None,
    "last_name_redacted": "robot_cc",
    "role_of_user": "Внешняя система",
    "commentaries": "ДС : житель  здоров;При осмотре течь не обнаружена.;ХВС открыта на квартиру и см бачок",
    "deffect_category_name": "Сантехника",
    "deffect_category_id": "1594",
    "deffect_category_name_eng": None,
    "deffect_name": "Перелив сливного бачка унитаза",
    "short_deffect_name": "Перелив сливного бачка унитаза",
    "deffect_id": "2077",
    "code_of_deffect": "def_gen_eaf_eeb_ebi_fdh_bgc_ced_fbe_cbh_bca_fja_eax",
    "need_for_revision": "Нет",
    "description": "Из сливного бачка  вода течет на пол и в чашу унитаза , нет возможности перекрыть воду",
    "presence_of_question": None,
    "urgency_category": "Аварийная",
    "urgency_category_eng": "emergency",
    "district": "ВАО",
    "district_code": "400",
    "hood": "Восточное Измайлово",
    "hood_code": "405",
    "adress_of_problem": "11-я Парковая улица, дом 1/89, корпус 2",
    "adress_unom": 18781,
    "porch": "1",
    "floor": "3",
    "flat_number": "16",
    "dispetchers_number": "ОДС №4-Восточное Измайлово",
    "owner_company": "ООО «РЭУ-29 РАЙОНА ВОСТОЧНОЕ ИЗМАЙЛОВО»",
    "serving_company": "ООО «РЭУ-29 РАЙОНА ВОСТОЧНОЕ ИЗМАЙЛОВО»",
    "performing_company": "921831.0",
    "inn_of_performing_company": "7719468873.0",
    "request_status_eng": "request_resolved",
    "reason_for_decline": None,
    "id_of_reason_for_decline": None,
    "reason_for_decline_of_org": None,
    "id_of_reason_for_decline_of_org": None,
    "id_work_type_done": "15420",
    "used_material": None,
    "guarding_events": "Течь локализована. Перекрыта вода на квартиру",
    "id_guarding_events": "18426",
    "efficiency": "Оказана консультация",
    "efficiency_eng": "consulted",
    "being_on_revision": "Нет",
    "alerted_feature": "Нет",
    "wanted_time_from": None,
    "wanted_time_to": None,
    "review": "Работы выполнены",
    "grade_for_service": "Хорошо",
    "grade_for_service_eng": "good",
    "payment_category": "бесплатная",
    "payment_category_eng": "free",
    "payed_by_card": "Нет",
#--------------------------------------
    "request_status": "Закрыта",
    "times_returned": 1,
    "work_type_done": "Оказана консультация с выходом специалиста на место",    # это они называют результат закрытия

    "closure_date": "2021-01-01T09:30:31.519054Z",
    "date_of_creation": "2021-01-01T09:07:25.519054Z",
    "date_of_start": "2021-01-03T11:05:21.519054Z",
    "date_of_last_return_for_revision": None,
    "date_of_review": "2021-04-15T22:43:28.519054Z",
    "date_of_previous_request_close": "2021-01-01T01:05:25.519054Z"
}

In [157]:
CONST_DEF_IDS_FOR_CASE_5 = list(pd.read_csv('application4.csv')['Корневой идентификатор'])
CODE_OF_DEFS_DONE_UNDER_10 = ["2303", "2245", "1903", "2396", "1922", "1771", "2096", "7907", "7906"]

times_allowed_df = pd.read_csv("application3.csv", index_col=False)[["Корневой идентификатор", "Время"]]
times = list(times_allowed_df["Время"])
ids = list(times_allowed_df["Корневой идентификатор"].astype(str))
TIMES_ALLOWED = {
    ids[i]: times[i] for i in range(len(ids))
}

In [158]:
class AnomalyDetector:
    def __init__(self):
        self.CONST_DEF_IDS_FOR_CASE_5 = CONST_DEF_IDS_FOR_CASE_5
        self.CODE_OF_DEFS_DONE_UNDER_10 = CODE_OF_DEFS_DONE_UNDER_10
        self.TIMES_ALLOWED = TIMES_ALLOWED
        self.EXCEPTION_DEFFECTS = [
                "Ввод в эксплуатацию ИПУ воды (замена, демонтаж, пропуск межповерочного интервала)",
                "Подача документов о поверке ИПУ воды в электронном виде",
            ]
        self.FEATURE_USED_FOR_TIME_BEGGINING = "date_of_creation"

    def __utils_parse_date(self, string):
        try:
            return np.datetime64(string) 
        except:
            return

    def get_time_allowed(self, deffect_id):
        time_allowed = self.TIMES_ALLOWED.get(deffect_id)
        if not time_allowed or time_allowed == '-':
            time_allowed = True
        else:
            time_allowed = int(time_allowed)
        return time_allowed

    def __case1(self, deffect_name, deffect_id, efficiency, time_previous_to_beggining, time_closure_to_beggining, time_revision_to_beggining, work_type_done, times_returned, can_be_done_in_under_10, grade_for_service):
        if not time_previous_to_beggining:   # если это не хотя бы 2 раз заявки от одного человека, то здесь это не аномалия
            return False

        time_allowed = self.get_time_allowed(deffect_id)

        if deffect_name not in self.EXCEPTION_DEFFECTS and efficiency != "Выполнено" and work_type_done != "Аварийное/плановое отключение" and time_previous_to_beggining <= time_allowed:
            return True
        return False

    def __case2(self, deffect_name, deffect_id, efficiency, time_previous_to_beggining, time_closure_to_beggining, time_revision_to_beggining, work_type_done, times_returned, can_be_done_in_under_10, grade_for_service):
        if not time_previous_to_beggining:   # если это не хотя бы 2 раз заявки от одного человека, то здесь это не аномалия
            return False
        
        time_allowed = self.get_time_allowed(deffect_id)

        if deffect_name not in self.EXCEPTION_DEFFECTS and efficiency == "Выполнено" and time_closure_to_beggining >= 10 and not times_returned and time_previous_to_beggining <= time_allowed:
            return True
        return False

    def __case3(self, deffect_name, deffect_id, efficiency, time_previous_to_beggining, time_closure_to_beggining, time_revision_to_beggining, work_type_done, times_returned, can_be_done_in_under_10, grade_for_service):
        if not time_previous_to_beggining:   # если это не хотя бы 2 раз заявки от одного человека, то здесь это не аномалия
            return False
        
        time_allowed = self.get_time_allowed(deffect_id)

        if can_be_done_in_under_10 and 10 >= time_closure_to_beggining and not times_returned and time_previous_to_beggining <= time_allowed:
            return True
        return False

    def __case4(self, deffect_name, deffect_id, efficiency, time_previous_to_beggining, time_closure_to_beggining, time_revision_to_beggining, work_type_done, times_returned, can_be_done_in_under_10, grade_for_service):
        if efficiency == "Выполнено" and 10 >= time_closure_to_beggining and not can_be_done_in_under_10:
            return True
        return False

    def __case5(self, deffect_name, deffect_id, efficiency, time_previous_to_beggining, time_closure_to_beggining, time_revision_to_beggining, work_type_done, times_returned, can_be_done_in_under_10, grade_for_service):
        if efficiency == "Выполнено" and not times_returned and deffect_id in self.CONST_DEF_IDS_FOR_CASE_5:
            return True
        return False

    def __case6(self, deffect_name, deffect_id, efficiency, time_previous_to_beggining, time_closure_to_beggining, time_revision_to_beggining, work_type_done, times_returned, can_be_done_in_under_10, grade_for_service):
        time_allowed = self.get_time_allowed(deffect_id)

        if not time_revision_to_beggining and time_previous_to_beggining and time_previous_to_beggining <= time_allowed:   # Если в заяке не указано, что у нее была предыдущая, но есть отдельной заявкой близкая по времени - она аномальная
            return True
        return False

    def __case7(self, deffect_name, deffect_id, efficiency, time_previous_to_beggining, time_closure_to_beggining, time_revision_to_beggining, work_type_done, times_returned, can_be_done_in_under_10, grade_for_service):
        if grade_for_service == "Неудовлетворительно" and efficiency == "Выполнено":
            return True
        return False

    def is_anomaly_function(self, dic):
            '''
            dic: one dict (function will not be used for list of dicts - it will be done in a cycle)

            returns: (bool: anomaly or not, [list of anomaly_cases])
            '''
            dic["closure_date"] = self.__utils_parse_date(dic["closure_date"])
            dic["date_of_creation"] = self.__utils_parse_date(dic["date_of_creation"])
            try:
                dic["date_of_start"] = self.__utils_parse_date(dic["date_of_start"])
            except:
                pass
            dic["date_of_last_return_for_revision"] = self.__utils_parse_date(dic["date_of_last_return_for_revision"])
            dic["date_of_review"] = self.__utils_parse_date(dic["date_of_review"])
            dic["date_of_previous_request_close"] = self.__utils_parse_date(dic["date_of_previous_request_close"])

            can_be_done_in_under_10 = False
            if dic["deffect_id"] in self.CODE_OF_DEFS_DONE_UNDER_10:
                can_be_done_in_under_10 = True
            
            time_closure_to_beggining = (dic["closure_date"] - dic[self.FEATURE_USED_FOR_TIME_BEGGINING]) / np.timedelta64(1, 'm')        # МИНУТЫ - Времени на закрытие текущей заявки(одной строчки)

            time_previous_to_beggining = None
            time_revision_to_beggining = None

            if dic["date_of_last_return_for_revision"]:
                time_previous_to_beggining = (dic["date_of_last_return_for_revision"] - dic[self.FEATURE_USED_FOR_TIME_BEGGINING]) / np.timedelta64(1, 'h')    # ЧАСЫ - Времени от предыдущей до создания текущей заявки
                time_revision_to_beggining = time_previous_to_beggining
            if dic["date_of_previous_request_close"]:
                time_previous_to_beggining = (dic[self.FEATURE_USED_FOR_TIME_BEGGINING] - dic["date_of_previous_request_close"]) / np.timedelta64(1, 'h')    # ЧАСЫ - Времени от предыдущей до создания текущей заявки

            if dic["request_status"] not in ["Закрыта через МАРМ", "Закрыта"]:
                return (False, [])

            list_of_anomaly_cases = []
            anomaly_checks = []
            cases = [
                self.__case1,
                self.__case2,
                self.__case3,
                self.__case4,
                self.__case5,
                self.__case6,
                self.__case7
            ]
            for i, case in enumerate(cases):
                check = case(
                    deffect_name=dic["deffect_name"],
                    deffect_id=dic["deffect_id"],
                    efficiency=dic["efficiency"],
                    time_previous_to_beggining=time_previous_to_beggining,
                    time_closure_to_beggining=time_closure_to_beggining,
                    time_revision_to_beggining=time_revision_to_beggining,
                    work_type_done=dic["work_type_done"],
                    times_returned=dic["times_returned"],
                    can_be_done_in_under_10=can_be_done_in_under_10,
                    grade_for_service=dic["grade_for_service"]
                )
                anomaly_checks.append(check)
                if check:
                    list_of_anomaly_cases.append(i + 1)
            return any(anomaly_checks), list_of_anomaly_cases

    def __call__(self, dic):
        return self.is_anomaly_function(dic)

---
# Анализ данных и обучение модели:

In [159]:
useful_features = [
    "date_of_creation",
    "date_of_start",
    "date_of_previous_request_close",
    "closure_date",
    "date_of_last_return_for_revision",
    "date_of_review",
    
    "commentaries",
    "deffect_category_name",
    "short_deffect_name",
    "deffect_name",
    "need_for_revision",
    "urgency_category",
    "description",
    "work_type_done",
    "guarding_events",
    "efficiency",
    "payment_category",
    "alerted_feature",
    "review",
    "grade_for_service",
    "times_returned"
    # если читать df из БД, то там будет еще поле "date_of_previous_request_close"
]

useful_dates = [
    "date_of_creation",
    "date_of_start",
    "date_of_previous_request_close",
    "closure_date",
    "date_of_last_return_for_revision",
    "date_of_review"
]

useful_features = ", ".join(useful_features)

In [160]:
useful_features

'date_of_creation, date_of_start, date_of_previous_request_close, closure_date, date_of_last_return_for_revision, date_of_review, commentaries, deffect_category_name, short_deffect_name, deffect_name, need_for_revision, urgency_category, description, work_type_done, guarding_events, efficiency, payment_category, alerted_feature, review, grade_for_service, times_returned'

In [161]:
import json
url_path = "./db_url.json"
with open(url_path) as json_file:
    dic = json.load(json_file)

url = dic["dialect+driver"] + "://" + dic["username"] + ":" + dic["password"] + "@" + dic["host"] + ":" + dic["port"] + "/" + dic["database"]

In [162]:
engine = sqlalchemy.create_engine(url)

In [163]:
def read_sql_data(query, engine, useful_features, useful_dates, chunksize=50000):
    with engine.connect().execution_options(autocommit=True) as conn:
        df_iter = pd.read_sql(
        query,
        con = conn,
        chunksize=chunksize,
        parse_dates=useful_dates
        )
    return df_iter

def iter_into_csv(df_iter, path="label_encoders_learner.csv.csv"):
    for df in df_iter:
        df.to_csv("label_encoders_learner.csv.csv", mode='a', index=False, header=False)

# Creation of label_encoders_learner.csv:

# df_iter = read_sql_data(
#     query = 
#     f"""
#     SELECT
#     {useful_features}
#     FROM requests
#     """,
#     engine=engine,
#     useful_features=useful_features,
#     useful_dates=useful_dates
# )

# iter_into_csv(df_iter)

# Creation of learning_data.csv:



In [164]:
import seaborn as sns
import matplotlib.pyplot as plt

In [165]:
# plt.figure(figsize=(10, 6))
# plt.xlabel('Grade for service', fontsize=15)
# plt.ylabel('Count', fontsize=15)
# plt.yticks(np.arange(0, 30000, 2000))
# sns.histplot(df["grade_for_service"])
# None

---

In [166]:
from sklearn.preprocessing import LabelEncoder

In [167]:
class CombinedModel():
    '''
    Класс комбинированной модели
    '''
    def __init__(self, detector, model, label_encoders, prob_threshold=0.3):
        '''
        detector: AnomalyDetector object
        label_encoders: dict of fitted LabelEncoder objects for each categorical column
        prob_threshold: prob threshold to classify prediction as anomaly for model
        '''
        self.prob_threshold = prob_threshold
        self.anomaly_detector = detector
        self.model = model  # этот аттрибут я не инкапсулирую, с ним я буду обращаться как с обычной моделькой из взятого фреймворка; это будет уже обученная модель; внутри класса она будет только предсказывать
        self.useful_features = [
            "date_of_creation",
            "date_of_previous_request_close",
            "closure_date",
            "date_of_last_return_for_revision",
            # "date_of_review",
            # "date_of_start",
            
            # "commentaries",
            # "deffect_category_name",
            # "short_deffect_name",
            "deffect_name",     # порядка 200 различных категорий, нанов нет
            "need_for_revision",    # 2 категории - без нанов
            "urgency_category",     # 2 категории - без нанов
            # "description",        # полностью текстовая фича
            # "work_type_done",     # 3300 различных значений - с этим попозже
            "guarding_events",      # 78 различных плюс нан (79 суммарно)
            "efficiency",       # 4 категории плюс нан
            "payment_category",     # 3 категории - без нанов
            "alerted_feature",      # 2 категории - без нанов
            "review",       # Здесь можно Выполнено / Не выполнено / Не указано(null) / Подробный комментарий - там всего 900 отзывов словами написаных из 35к не null-ов
            "grade_for_service",    # наны говорим, что без оценки
            "times_returned"    # наны говорим, что нули
        ]

        self.useful_dates = [
            # "date_of_start",
            # "date_of_review",
            "date_of_creation",
            "date_of_previous_request_close",
            "date_of_last_return_for_revision",
            "closure_date"
        ]

        self.categorical_features = [
            "deffect_name",     # порядка 200 различных категорий, нанов нет
            "need_for_revision",    # 2 категории - без нанов
            "urgency_category",     # 2 категории - без нанов
            # "description",        # полностью текстовая фича
            # "work_type_done",     # 3300 различных значений - с этим попозже
            "guarding_events",      # 78 различных плюс нан (79 суммарно)
            "efficiency",       # 4 категории плюс нан
            "payment_category",     # 3 категории - без нанов
            "alerted_feature",      # 2 категории - без нанов
            "review",       # Здесь можно Выполнено / Не выполнено / Не указано(null) / Подробный комментарий - там всего 900 отзывов словами написаных из 35к не null-ов
            "grade_for_service",    # наны говорим, что без оценки 
        ]

        self.numerical_features = [
            "time_previous_to_creation",
            "time_revision_to_creation",
            "time_closure_to_creation",
            "times_returned"
        ]

        self.types = {
            "date_of_creation": 'datetime64[ns]',
            # "date_of_start": np.datetime64,
            "date_of_last_return_for_revision": 'datetime64[ns]',
            "date_of_previous_request_close": 'datetime64[ns]',
            "closure_date": 'datetime64[ns]',
            "times_returned": int
        }

        self.label_encoders = label_encoders    # зафичены на кат фичах по 840100 строчек из БД - если будем добавлять в бд строчки, надо будет делать другую обучающую .csv для них

    def _dates_to_times(self, df):
        '''
        Creates columns for time ranges and drops date columns
        '''
        df = df.astype(self.types)
        df["time_closure_to_creation"] = (df["closure_date"].astype('datetime64[ns]') - df["date_of_creation"].astype('datetime64[ns]')) / np.timedelta64(1, 'm')

        condition = (df["date_of_last_return_for_revision"].notna())
        df["date_of_last_return_for_revision"] = df["date_of_last_return_for_revision"].where(condition, -1, inplace=False)
        df.loc[df["date_of_last_return_for_revision"] == -1, "date_of_last_return_for_revision"] = df.loc[df["date_of_last_return_for_revision"] == -1, "date_of_creation"]     # если там None стоит в значении, то мы говорим, что оно = creation, чтобы 0 получить по длительности

        condition = (df["date_of_previous_request_close"].notna())
        df["date_of_previous_request_close"] = df["date_of_previous_request_close"].where(condition, -1, inplace=False)
        df.loc[df["date_of_previous_request_close"] == -1, "date_of_previous_request_close"] = df.loc[df["date_of_previous_request_close"] == -1, "date_of_creation"]

        df["time_previous_to_creation"] = (df["date_of_creation"].astype('datetime64[ns]') - df["date_of_previous_request_close"].astype('datetime64[ns]')) / np.timedelta64(1, 'h')
        df["time_revision_to_creation"] = (df["date_of_last_return_for_revision"].astype('datetime64[ns]') - df["date_of_creation"].astype('datetime64[ns]')) / np.timedelta64(1, 'h')

        df = df.drop(self.useful_dates, axis="columns")
        return df

    def _nan_filler(self, df):
        '''
        fills nans
        '''
        df["guarding_events"] = df["guarding_events"].fillna("Не указано")
        df["grade_for_service"] = df["grade_for_service"].fillna("Не указано")
        df["efficiency"] = df["efficiency"].fillna("Не указано")
        df["review"] = df["review"].fillna("Не указано")
        df.loc[(df["review"] != 'Работы выполнены') & (df['review'] != 'Работы не выполнены') & (df['review'] != "Не указано"), "review"] = "Подробный комментарий"
        return df

    def _label_encode_cat_feats(self, df):
        '''
        label encoding of categorical features for model embedding
        '''
        for feature, label_encoder in self.label_encoders.items():
            df[feature] = label_encoder.transform(df[feature])
        return df

    def _make_input_model_passable(self, list_of_dics_or_dic, already_df=False):
        '''

        Метод, который преобразует входной json к предобработанному pd.DataFrame

        '''
        if not already_df:
            try:    # многомерный
                df = pd.DataFrame(list_of_dics_or_dic)[self.useful_features]
            except: # одномерный
                df = pd.DataFrame(list_of_dics_or_dic, index=[0])[self.useful_features]
        else:
            df = list_of_dics_or_dic[self.useful_features]
        df["times_returned"] = df["times_returned"].fillna(0)
        df = df.astype(self.types)
        df = self._dates_to_times(df)
        df = self._nan_filler(df)
        df = self._label_encode_cat_feats(df)

        return df

    def _df_into_x_cat_x_num(self, list_of_dics_or_dic):
        '''
        Takes df or list_of_dics_or_dic before _make_input_model_passable;

        returns: (X_cat, X_num), которые уже подаются на вход модели
        '''
        df = self._make_input_model_passable(list_of_dics_or_dic)
        X_cat = df[self.categorical_features].values
        X_num = df[self.numerical_features].values
        return torch.tensor(X_cat, dtype=torch.int64), torch.tensor(X_num, dtype=torch.float32)

    def _check_single_if(self, dic):
        '''
        Checks ifs for one dic
        '''
        is_anomaly, anomaly_cases = self.anomaly_detector(dic)
        if is_anomaly:
            return {
                "root_id": dic["root_id"],
                "is_anomaly": is_anomaly,
                "anomaly_cases": anomaly_cases
            }
        return {
                "root_id": dic["root_id"],
                "is_anomaly": is_anomaly,
                "anomaly_cases": []
            }
    
    def _get_model_prediction(self, X_cat, X_num):
        '''
        prob_threshold - вероятность - после которой считаем, что это уже аномалия
        '''
        pred = self.model(X_cat, X_num)

        net_probability = torch.sigmoid(pred)

        pred = (net_probability >= self.prob_threshold).to(int)

        return pred, net_probability

    # Следующие два метода нужны для работы уже внутри сервиса:
    def predict_single(self, dic):
        '''
        Takes: single dict, returns: single dict
        '''
        dic = dic.copy()
        single_if_result = self._check_single_if(dic)
        self.model.eval()
        X_cat, X_num = self._df_into_x_cat_x_num(dic)
        prediction, net_probability = self._get_model_prediction(X_cat, X_num)
        single_if_result["net_probability"] = None
        if net_probability >= 0.1:
            single_if_result["net_probability"] = round(net_probability.item() * 100)

        return single_if_result

    def predict_multiple(self, list_of_dics):
        '''
        Takes: list of dicts, returns: list of dicts
        Всегда применяет модель к инпутам
        '''
        result = []

        try:
            self.model.eval()
            X_cat, X_num = self._df_into_x_cat_x_num(list_of_dics)
            predictions, net_probabilitys = self._get_model_prediction(X_cat, X_num)
        except:
            net_probabilitys = [0] * len(list_of_dics)

        for i, dic in enumerate(list_of_dics):
            dic = dic.copy()
            single_if_result = self._check_single_if(dic)
            # print(single_if_result)
            single_if_result["net_probability"] = None
            if net_probabilitys[i] >= 0.1:
                single_if_result["net_probability"] = round(net_probabilitys[i].item() * 100)


            result.append(single_if_result)
            # print(single_if_result)
        # print(
        #     self._make_input_model_passable(list_of_dics).info()
        # )

        return result

In [168]:
supportive_comb_model = CombinedModel(
    detector=AnomalyDetector(),
    model=None,
    label_encoders=None
)

In [169]:
df_categorical = supportive_comb_model._nan_filler(pd.read_csv("label_encoders_learner.csv", low_memory=False, usecols=supportive_comb_model.categorical_features))

**Creation of dict of label encoders to pass to the model:**

In [170]:
label_encoders = {}
for feature in supportive_comb_model.categorical_features:
    label_encoders[feature] = LabelEncoder()

for feature, label_encoder in label_encoders.items():
    df_categorical[feature] = label_encoder.fit_transform(df_categorical[feature])

In [171]:
# Actual model for prediction is this one:
detector = AnomalyDetector()

comb_model = CombinedModel(
    detector=AnomalyDetector(),
    model=None,
    label_encoders=label_encoders
)

**SQL запросом получил выборку для обучения и теста модели:**

In [172]:
from sklearn.utils import shuffle

In [173]:
learning_df = pd.read_csv("learning_rare.csv", parse_dates=supportive_comb_model.useful_dates)
learning_df.is_anomaly = learning_df.is_anomaly.astype(int)
learning_df = shuffle(learning_df)

In [174]:

Y = learning_df['is_anomaly']
X = comb_model._make_input_model_passable(learning_df)

In [175]:
from sklearn.model_selection import train_test_split

X_train, X_val, Y_train, Y_val = train_test_split(X, Y, test_size=0.25, shuffle=True)

In [176]:
def fit_epoch(model, train_loader, criterion, optimizer, scheduler, scaler=None, prob_threshold=0.5):
    '''

    Функция обучения по всем батчам 1 раз (1 эпоха)

    scaler: gradient scaler from torch.amp, попозже добавлю обучение с ним

    '''
    model.train()

    running_loss = 0.0
    running_acc = 0
    processed_size = 0
    processed_batches = 0

    for X_cat, X_num, labels in train_loader:
        optimizer.zero_grad()

        X_cat = X_cat.to(CFG.device)
        X_num = X_num.to(CFG.device)
        labels = labels.to(CFG.device).unsqueeze(1).to(torch.float32)

        outputs = model(X_cat, X_num)
        loss = criterion(outputs, labels)

        loss.backward()
        optimizer.step()

        if scheduler:
            scheduler.step()

        running_loss += loss.item() * labels.size(0)    # при очень большом размере батча последние два батча будут например размера 128 вместо 256, поэтому просто умножать на батч сайз неправильно, могут быть другого размера
        processed_size += labels.size(0)
        processed_batches += 1

        correctly_guessed = ((torch.sigmoid(outputs) >= prob_threshold).to(torch.int) == labels).sum().to(int)
        running_acc += correctly_guessed / labels.size(0)

    train_loss = running_loss / processed_size
    accuracy = running_acc / processed_batches
    
    return train_loss, accuracy

In [177]:
def eval_epoch(model, val_loader, criterion, prob_threshold=0.5):
    '''

    Одна эпоха по val выборке

    '''

    model.eval()
    
    running_loss = 0.0
    running_acc = 0
    processed_size = 0
    processed_batches = 0

    for X_cat, X_num, labels in val_loader:

        X_cat = X_cat.to(CFG.device)
        X_num = X_num.to(CFG.device)
        labels = labels.to(CFG.device).unsqueeze(1).to(torch.float32)

        with torch.no_grad():
            outputs = model(X_cat, X_num)
            loss = criterion(outputs, labels)

        running_loss += loss.item() * labels.size(0)    # при очень большом размере батча последние два батча будут например размера 128 вместо 256, поэтому просто умножать на батч сайз неправильно, могут быть другого размера
        processed_size += labels.size(0)
        processed_batches += 1

        correctly_guessed = ((torch.sigmoid(outputs) >= prob_threshold).to(torch.int) == labels).sum().to(int)
        running_acc += correctly_guessed / labels.size(0)
    
    val_loss = running_loss / processed_size
    accuracy = running_acc / processed_batches

    return val_loss, accuracy

In [178]:
def train(train_loader, val_loader, model, optimizer, scheduler=None, epochs=10, scaler=None, criterion=nn.BCEWithLogitsLoss(), show_3D_quality=False, prob_threshold=0.2):
    '''

    Full train and validation cycle for model
    
    '''

    history = []
    log_template = "\nEpoch {ep:03d} train_loss: {t_loss:0.4f} t_acc:{t_acc:0.4f} val_loss: {v_loss:0.4f} v_acc:{v_acc:0.4f}"

    with tqdm(desc="epoch", total=epochs) as pbar_outer:

        for epoch in range(epochs):
            train_loss, train_acc = fit_epoch(model, train_loader, criterion, optimizer, scheduler, scaler, prob_threshold=prob_threshold)
            val_loss, val_acc = eval_epoch(model, val_loader, criterion, prob_threshold=prob_threshold)
            history.append((train_loss, val_loss))
            
            pbar_outer.update(1)
            tqdm.write(log_template.format(ep=epoch + 1, t_loss=train_loss, t_acc=train_acc, v_loss=val_loss, v_acc=val_acc))
            
    return history

In [179]:
class CatAndNumModel(nn.Module):
    def __init__(self, embedding_sizes, n_numeric):
        super().__init__()
        self.embeddings = nn.ModuleList([nn.Embedding(categories, size) for categories, size in embedding_sizes])

        n_emb = sum(emb.embedding_dim for emb in self.embeddings) # length of all embeddings combined

        self.n_emb, self.n_cont = n_emb, n_numeric

        self.BN1 = nn.BatchNorm1d(self.n_cont)

        self.FC = nn.Sequential(
            nn.Linear(self.n_emb + self.n_cont, 64),
            nn.ReLU(),
            nn.BatchNorm1d(64),

            nn.Linear(64, 128),
            nn.ReLU(),
            nn.BatchNorm1d(128),

            nn.Linear(128, 128),
            nn.ReLU(),
            
            nn.Linear(128, 1)
        )
        

    def forward(self, x_cat, x_num):
        x = [e(x_cat[:, i]) for i, e in enumerate(self.embeddings)]
        x = torch.cat(x, 1)
        # x = self.emb_drop(x)

        x2 = self.BN1(x_num)
        x = torch.cat([x, x2], 1)

        x = self.FC(x)

        return x

class NumModel(nn.Module):
    def __init__(self, embedding_sizes, n_numeric):
        super().__init__()

        self.n_cat, self.n_cont = len(embedding_sizes), n_numeric

        self.BN1 = nn.BatchNorm1d(self.n_cont + self.n_cat)

        self.FC = nn.Sequential(
            nn.Linear(self.n_cont, 256),
            nn.ReLU(),
            nn.BatchNorm1d(256),

            nn.Linear(256, 256),
            nn.ReLU(),
            nn.BatchNorm1d(256),

            nn.Linear(256, 256),
            nn.ReLU(),
            
            nn.Linear(256, 1)
        )

    def forward(self, x_cat, x_num):
        x = torch.concat([x_cat, x_num], dim=1)

        # x = self.BN1(x)

        x = self.FC(x)

        return x

In [180]:
class CatNumDataset(Dataset):
    def __init__(self, X_cat, X_num, Y):
        super().__init__()
        self.X_cat = X_cat # categorical columns
        self.X_num = X_num # numerical columns
        self.Y = Y
        
    def __len__(self):
        return len(self.Y)
    
    def __getitem__(self, idx):
        return self.X_cat[idx], self.X_num[idx], self.Y[idx]

In [181]:
categories_count = df_categorical.nunique().to_dict()       # В каких фичах сколько уникальных значений
column_names_to_embed = list(categories_count.keys())
embedding_sizes = [(unique_cats, min(50, (unique_cats + 1) // 2)) for _, unique_cats in categories_count.items()]    # список из (unique_amount, embed_dim)

In [182]:
train_data = CatNumDataset(
    X_cat=X_train.loc[:, column_names_to_embed].values.astype(np.int64),
    X_num=X_train.drop(columns=column_names_to_embed).values.astype(np.float32),
    Y=Y_train.values
)

val_data = CatNumDataset(
    X_cat=X_val.loc[:, column_names_to_embed].values.astype(np.int64),
    X_num=X_val.drop(columns=column_names_to_embed).values.astype(np.float32),
    Y=Y_val.values
)

train_loader = DataLoader(train_data, batch_size=CFG.batch_size, shuffle=True)
val_loader = DataLoader(val_data, batch_size=CFG.batch_size, shuffle=False)

In [183]:
model = CatAndNumModel(embedding_sizes=embedding_sizes, n_numeric=4)
optimizer = optim.Adam(model.parameters(), lr=4e-3, betas=(0.9, 0.999), weight_decay=0.1)

# scheduler.step нужно первый раз делать обязательно после optimizer.step, потому что иначе мы просто пропустим первый шаг scheduler
exp_scheduler = lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.95)

Вариант модели без эмбеддингов:

In [184]:
# model = NumModel(embedding_sizes=embedding_sizes, n_numeric=4)
# optimizer = optim.Adam(model.parameters(), lr=4e-2, betas=(0.9, 0.999), weight_decay=0.01)

# # scheduler.step нужно первый раз делать обязательно после optimizer.step, потому что иначе мы просто пропустим первый шаг scheduler
# exp_scheduler = lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.95)

In [185]:
history = train(
    train_loader=train_loader,
    val_loader=val_loader,
    model=model.to(CFG.device),
    optimizer=optimizer,
    scheduler=exp_scheduler,

    epochs=10,

    prob_threshold=0.5      # Влияет только на отображение accuracy, лучше вообще и
)

epoch:  10%|█         | 1/10 [00:01<00:17,  1.92s/it]


Epoch 001 train_loss: 0.4571 t_acc:0.8629 val_loss: 0.4627 v_acc:0.9342


epoch:  20%|██        | 2/10 [00:03<00:14,  1.87s/it]


Epoch 002 train_loss: 0.2114 t_acc:0.9323 val_loss: 0.4092 v_acc:0.9342


epoch:  30%|███       | 3/10 [00:05<00:12,  1.79s/it]

...


Epoch 008 train_loss: 0.1369 t_acc:0.9664 val_loss: 0.1739 v_acc:0.9697


epoch:  90%|█████████ | 9/10 [00:15<00:01,  1.69s/it]


Epoch 009 train_loss: 0.1355 t_acc:0.9664 val_loss: 0.1473 v_acc:0.9666


epoch: 100%|██████████| 10/10 [00:17<00:00,  1.73s/it]


Epoch 010 train_loss: 0.1291 t_acc:0.9693 val_loss: 0.1771 v_acc:0.9730


---
# Testing:

In [186]:
# Actual model for prediction is this one:
detector = AnomalyDetector()

comb_model = CombinedModel(
    detector=AnomalyDetector(),
    model=model.to('cpu'),
    label_encoders=label_encoders,

    prob_threshold=0.5
)

In [187]:
l = [test_dic.copy(), test_dic.copy(), test_dic.copy()]

In [188]:
comb_model.predict_single(test_dic.copy())

/var/folders/7j/pb93vgwj3xq45tf5_g4_z9r00000gp/T/ipykernel_2566/3972226764.py:14: DeprecationWarning: parsing timezone aware datetimes is deprecated; this will raise an error in the future
  return np.datetime64(string)


IndexError: index out of range in self

In [ ]:
comb_model.predict_multiple(l.copy())

/var/folders/7j/pb93vgwj3xq45tf5_g4_z9r00000gp/T/ipykernel_2566/3972226764.py:14: DeprecationWarning: parsing timezone aware datetimes is deprecated; this will raise an error in the future
  return np.datetime64(string)


[{'root_id': '73417411',
  'is_anomaly': True,
  'anomaly_cases': [1],
  'net_probability': None},
 {'root_id': '73417411',
  'is_anomaly': True,
  'anomaly_cases': [1],
  'net_probability': None},
 {'root_id': '73417411',
  'is_anomaly': True,
  'anomaly_cases': [1],
  'net_probability': None}]

In [ ]:
testing_list_of_dics = pd.read_csv("bug_fixer.csv").astype("object").to_dict("records")

/var/folders/7j/pb93vgwj3xq45tf5_g4_z9r00000gp/T/ipykernel_2566/816571482.py:1: DtypeWarning: Columns (45,62) have mixed types. Specify dtype option on import or set low_memory=False.
  testing_list_of_dics = pd.read_csv("bug_fixer.csv").astype("object").to_dict("records")


In [ ]:
comb_model.predict_single(testing_list_of_dics[0])

{'root_id': 76746679,
 'is_anomaly': False,
 'anomaly_cases': [],
 'net_probability': None}

In [ ]:
(np.datetime64('2020-12-31 22:05:24.918411') - np.datetime64('2019-12-31 22:05:24.918411')) / np.timedelta64(1, 'm')

527040.0

In [ ]:
%%time
comb_model.predict_multiple(testing_list_of_dics.copy())

CPU times: user 11 s, sys: 863 ms, total: 11.9 s
Wall time: 10.3 s


[{'root_id': 76746679,
  'is_anomaly': False,
  'anomaly_cases': [],
  'net_probability': None},
 {'root_id': 76746684,
  'is_anomaly': False,
  'anomaly_cases': [],
  'net_probability': 15},
 {'root_id': 76746686,
  'is_anomaly': False,
  'anomaly_cases': [],
  'net_probability': 12},
 {'root_id': 76746691,
  'is_anomaly': False,
  'anomaly_cases': [],
  'net_probability': None},
 {'root_id': 76746693,
  'is_anomaly': False,
  'anomaly_cases': [],
  'net_probability': 21},
 {'root_id': 76746705,
  'is_anomaly': False,
  'anomaly_cases': [],
  'net_probability': None},
 {'root_id': 76746708,
  'is_anomaly': False,
  'anomaly_cases': [],
  'net_probability': 64},
 {'root_id': 76746710,
  'is_anomaly': False,
  'anomaly_cases': [],
  'net_probability': 68},
 {'root_id': 76746726,
  'is_anomaly': False,
  'anomaly_cases': [],
  'net_probability': 11},
 {'root_id': 76746728,
  'is_anomaly': False,
  'anomaly_cases': [],
  'net_probability': None},
 {'root_id': 76746733,
  'is_anomaly': Fal

---
# Сохранение пиклов:

In [ ]:
detector_pickle = open('detector.pickle', 'wb')
dill.dump(detector, detector_pickle)
detector_pickle.close()

In [ ]:
comb_model_pickle = open('comb_model.pickle', 'wb')
dill.dump(comb_model, comb_model_pickle)
comb_model_pickle.close()

---

In [ ]:
test_dic = {
    "root_id": "73417411",
    "version_id": "73509990",
    "request_number": "00002335/21",
    "mos_ru_request_number": None,
    "name_of_source": "телефон",
    "name_of_source_eng": "phone",
    "name_of_creator": "frontline",
    "incident_feature": "Нет",
    "root_identificator_of_maternal": None,
    "number_of_maternal": None,
    "last_name_redacted": "robot_cc",
    "role_of_user": "Внешняя система",
    "commentaries": "ДС : житель  здоров;При осмотре течь не обнаружена.;ХВС открыта на квартиру и см бачок",
    "deffect_category_name": "Сантехника",
    "deffect_category_id": "1594",
    "deffect_category_name_eng": None,
    "deffect_name": "Перелив сливного бачка унитаза",
    "short_deffect_name": "Перелив сливного бачка унитаза",
    "deffect_id": "2077",
    "code_of_deffect": "def_gen_eaf_eeb_ebi_fdh_bgc_ced_fbe_cbh_bca_fja_eax",
    "need_for_revision": "Нет",
    "description": "Из сливного бачка  вода течет на пол и в чашу унитаза , нет возможности перекрыть воду",
    "presence_of_question": None,
    "urgency_category": "Аварийная",
    "urgency_category_eng": "emergency",
    "district": "ВАО",
    "district_code": "400",
    "hood": "Восточное Измайлово",
    "hood_code": "405",
    "adress_of_problem": "11-я Парковая улица, дом 1/89, корпус 2",
    "adress_unom": 18781,
    "porch": "1",
    "floor": "3",
    "flat_number": "16",
    "dispetchers_number": "ОДС №4-Восточное Измайлово",
    "owner_company": "ООО «РЭУ-29 РАЙОНА ВОСТОЧНОЕ ИЗМАЙЛОВО»",
    "serving_company": "ООО «РЭУ-29 РАЙОНА ВОСТОЧНОЕ ИЗМАЙЛОВО»",
    "performing_company": "921831.0",
    "inn_of_performing_company": "7719468873.0",
    "request_status_eng": "request_resolved",
    "reason_for_decline": None,
    "id_of_reason_for_decline": None,
    "reason_for_decline_of_org": None,
    "id_of_reason_for_decline_of_org": None,
    "id_work_type_done": "15420",
    "used_material": None,
    "guarding_events": "Течь локализована. Перекрыта вода на квартиру",
    "id_guarding_events": "18426",
    "efficiency": "Оказана консультация",
    "efficiency_eng": "consulted",
    "being_on_revision": "Нет",
    "alerted_feature": "Нет",
    "wanted_time_from": None,
    "wanted_time_to": None,
    "review": "Работы выполнены",
    "grade_for_service": "Хорошо",
    "grade_for_service_eng": "good",
    "payment_category": "бесплатная",
    "payment_category_eng": "free",
    "payed_by_card": "Нет",
#--------------------------------------
    "request_status": "Закрыта",
    "times_returned": None,
    "work_type_done": "Оказана консультация с выходом специалиста на место",    # это они называют результат закрытия

    "closure_date": "2021-01-01T09:30:31.519054Z",
    "date_of_creation": "2021-01-01T09:07:25.519054Z",
    "date_of_start": "2021-01-03T11:05:21.519054Z",
    "date_of_last_return_for_revision": None,
    "date_of_review": "2021-04-15T22:43:28.519054Z",
    "date_of_previous_request_close": "2021-01-01T00:07:25.519054Z"
}

In [ ]:
comb_model.predict_single(test_dic)

/var/folders/7j/pb93vgwj3xq45tf5_g4_z9r00000gp/T/ipykernel_2566/3972226764.py:14: DeprecationWarning: parsing timezone aware datetimes is deprecated; this will raise an error in the future
  return np.datetime64(string)


{'root_id': '73417411',
 'is_anomaly': True,
 'anomaly_cases': [1],
 'net_probability': None}